In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import os
# Import the API key.
from dotenv import load_dotenv
load_dotenv()
gm_api = os.getenv('google_maps_api')

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Angoche,61.9500,159.2333,21.24,63,100,27.20,RU,2022-01-31 23:42:00
1,1,Ponta Do Sol,32.6667,-17.1000,66.00,83,32,12.95,PT,2022-01-31 23:44:38
2,2,Priargunsk,50.3691,119.1012,-29.15,100,53,0.38,RU,2022-01-31 23:46:02
3,3,San Cristobal,7.7669,-72.2250,75.38,81,81,2.55,VE,2022-01-31 23:42:24
4,4,Kapaa,22.0752,-159.3190,76.12,71,0,1.01,US,2022-01-31 23:42:06


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=gm_api)

In [11]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,San Cristobal,7.7669,-72.2250,75.38,81,81,2.55,VE,2022-01-31 23:42:24
4,4,Kapaa,22.0752,-159.3190,76.12,71,0,1.01,US,2022-01-31 23:42:06
8,8,Bengkulu,-3.8004,102.2655,75.85,86,100,3.87,ID,2022-01-31 23:45:58
11,11,Atuona,-9.8000,-139.0333,78.46,77,14,18.12,PF,2022-01-31 23:46:05
22,22,Atar,20.5169,-13.0499,80.06,13,100,8.19,MR,2022-01-31 23:44:39
23,23,Saint-Philippe,-21.3585,55.7679,78.39,80,100,14.81,RE,2022-01-31 23:46:09
24,24,Nelson Bay,-32.7167,152.1500,85.28,74,10,1.99,AU,2022-01-31 23:46:09
25,25,Isangel,-19.5500,169.2667,82.51,89,100,5.75,VU,2022-01-31 23:46:09
28,28,Rikitea,-23.1203,-134.9692,78.01,67,66,4.72,PF,2022-01-31 23:46:10
29,29,Hithadhoo,-0.6000,73.0833,82.11,75,26,11.68,MV,2022-01-31 23:46:11


In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,San Cristobal,VE,75.38,7.7669,-72.2250,
4,Kapaa,US,76.12,22.0752,-159.3190,
8,Bengkulu,ID,75.85,-3.8004,102.2655,
11,Atuona,PF,78.46,-9.8000,-139.0333,
22,Atar,MR,80.06,20.5169,-13.0499,
23,Saint-Philippe,RE,78.39,-21.3585,55.7679,
24,Nelson Bay,AU,85.28,-32.7167,152.1500,
25,Isangel,VU,82.51,-19.5500,169.2667,
28,Rikitea,PF,78.01,-23.1203,-134.9692,
29,Hithadhoo,MV,82.11,-0.6000,73.0833,


In [19]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gm_api
}

In [18]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gm_api,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uECdnCrdpZ0RrXaydc4d7cWSGMK2YjOWCfeX7wufQiASI6uLc1ugPXBSlwuvKxyC8dzPMwhJ5FR4-Ia0oxbnQGXgHkLDhpK-v3Cwy1rg8erBSemFtPvXKMp8ti9tQMpOrXBD4B_uRXf6KnQVb9EF0I3Apf8QcPdBBjUrEcYyfmR4kFqbirM52j-O6iHRS4hpTGA-xYVKMYKmyyaJ3DA0W_EOTD5yRMgGGp_8d5wf36tPdZVf45FrJ8J_KL0lN9udWKjAMTcx6NijHy02vcmuY16SNdcuWod118o1oHYnBviptoCQju-VKECqVduFUIWhlxi0tFqaZnhiXw8qc4Kr8zUWu1bd9EekkjcxbWD4IRgMg53OAV-LNDhkrUqke4p6Y7ciQC3dWfhvMDCNECJLC0h0oYLDv5r5VNc_NEG0ZUVpWTNIuUFL_a7g',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [24]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,San Cristobal,VE,75.38,7.7669,-72.2250,Pirineos
4,Kapaa,US,76.12,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Bengkulu,ID,75.85,-3.8004,102.2655,Grage Hotel Bengkulu
11,Atuona,PF,78.46,-9.8000,-139.0333,Villa Enata
22,Atar,MR,80.06,20.5169,-13.0499,Oumou Elghoura
...,...,...,...,...,...,...
522,Brownsville,US,77.11,25.9018,-97.4975,Best Western Hotel Plaza Matamoros
524,Nueva Germania,PY,81.19,-23.9000,-56.5667,Estancia Ña Luisa
534,Boyolangu,ID,77.25,-8.1181,111.8935,Yos ega cv
535,Antalaha,MG,75.49,-14.9003,50.2788,Hotel Vitasoa


In [26]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [29]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))